**Resources** 

[Blurr Seq Classifier](https://colab.research.google.com/github/ohmeow/blurr/blob/master/nbs/01_modeling-core.ipynb#scrollTo=dhrIbw4cde0S)

[Fastai + Start Logging with W&B](https://docs.wandb.ai/guides/integrations/fastai)

[Visualize, Track & Compare Fastai Models](https://wandb.ai/borisd13/demo_config/reports/Visualize-Track-Compare-Fastai-Models--Vmlldzo4MzAyNA)

In [ ]:
!nvidia-smi

# Env setup

In [1]:
# !pip uninstall -y tensorflow torch
# !pip install -Uqq torch==1.10.2+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html 
# !pip install torch==1.7.1

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0


In [2]:
!pip install -Uqq transformers transformers['sentencepiece'] datasets wandb

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 325 kB 44.7 MB/s 
     |████████████████████████████████| 1.7 MB 33.3 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
     |████████████████████████████████| 6.5 MB 34.6 MB/s 
     |████████████████████████████████| 134 kB 34.5 MB/s 
     |████████████████████████████████| 1.1 MB 14.9 MB/s 
     |████████████████████████████████| 212 kB 42.5 MB/s 
     |████████████████████████████████| 127 kB 45.1 MB/s 
     |████████████████████████████████| 181 kB 45.7 MB/s 
     |████████████████████████████████| 144 kB 47.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 144 kB 44.2 MB/s 
     |████████████████████████████████| 271 kB 48.0 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |███████████████

In [ ]:
!pip install -Uqq git+https://github.com/ohmeow/blurr.git@dev-2.0.0 

In [2]:
import os
import wandb
import datasets
from transformers import *

from fastai.callback.wandb import *
from fastai.text.all import *
from fastai.callback.hook import _print_shapes

from blurr.utils import *
from blurr.data.core import *
from blurr.modeling.core import *

logging.set_verbosity_error()

In [15]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
# torch.cuda.set_device(1)
# print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")

RuntimeError: ignored

# Get Dataset

In [3]:
raw_datasets = datasets.load_dataset("rajeshradhakrishnan/malayalam_news")
raw_datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset malayalam_news downloaded and prepared to /root/.cache/huggingface/datasets/rajeshradhakrishnan___malayalam_news/malayalam_news/1.0.0/7d6fb88a050b410d95b4f750165a6f797581be76cc65793eb4c6b71ab9ca9b09. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4800
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 600
    })
})

In [4]:
raw_datasets['train'].features["label"].names

['business', 'entertainment', 'sports', 'technology']

In [5]:
raw_datasets['train'] = raw_datasets['train'].add_column("is_valid", [False] * len(raw_datasets['train']))
raw_datasets['test'] = raw_datasets['test'].add_column("is_valid", [True] * len(raw_datasets['test']))

In [6]:
final_ds = datasets.concatenate_datasets([raw_datasets['train'].shuffle().select(range(1000)), raw_datasets['test'].shuffle().select(range(200))])
mlnews_df = pd.DataFrame(final_ds)
mlnews_df.head()

,text,label,is_valid
0,"കൂടുതൽ സ്ഥലലഭ്യത നൽകുന്ന പുഷ് ബാക്ക് സീറ്റുകൾ , ഓരോ നാല് സീറ്റുകൾ വീതവും ടിവി മോണിറ്റർ സൌകര്യം , മികച്ച ഇന്ധനക്ഷമത എല്ലാറ്റിനുമുപരി സമയലാഭം . ഒറ്റനോട്ടത്തിൽ തന്നെ ആരുമൊന്ന് യാത്ര ചെയ്യാൻ ആഗ്രഹിക്കുന്ന ചന്തമുണ്ട് സ്വീഡിഷ് കമ്പനിയായ സ്കാനിയയുടെ ആഡംബര ബസ്സിന് . റിസർവേഷൻ തുടങ്ങാത്തതിനാൽ ആളുകളെല്ലാം അറിഞ്ഞെത്തുന്നതെയുള്ളൂ . എങ്കിലും കിട്ടുന്ന വണ്ടിയ്ക്ക് നാട്ടിലെത്താമെന്ന് വിചാരിച്ച് വന്നവർക്ക് അപ്രതീക്ഷിതമായി കേരള ആർടിസിയുടെ സ്കാനിയ കണ്ടപ്പോൾ ഉത്സാഹം . കർണാടക ആർടിസിയും , സ്വകാര്യ ബസ് ഓപ്പറേറ്റർമാരും നേരത്തെ തന്നെ നിരത്തിറക്കിയ സ്കാനിയയെ മൂന്ന് മാസത്തെ പരീക്ഷണ ഓട്ടം വിജയകരമായതിന്റെ ഉറപ്പിലാണ് സ...",0,False
1,"എന്നാൽ ഇന്ത്യൻ ക്രിക്കറ്റ് താരം ജസ്പ്രീത് ബുംറയാണ് പുതിയ വിവദത്തിലെ നായകൻ . താനൊരു ക്രിക്കറ്റ് പ്രേമിയാണെന്നും . ജസ്പ്രീത് ഭുംറയുടെ ബൌളിങ് ആണ് തന്നെ ക്രിക്കറ്റിലേക്ക് ആകർഷിക്കുന്നതെന്നും റാഷി ഖന്ന വിവിധ അഭിമുഖങ്ങളിലായി പറഞ്ഞിരുന്നു . തുടർന്ന് ഇരുവരും കടുത്ത പ്രണയത്തിലാണെന്നും പലപ്പോഴും ഇവർ തമ്മിൽ ഡേറ്റിങ് നടത്താറുണ്ടെന്നും വാർത്തകൾ വന്നു . എന്നാൽ അടുത്തിടെ ഒരു ചാറ്റ് ഷോയിൽ ഞെട്ടിക്കുന്ന പ്രതികരണം നടത്തിയിരിക്കുകയാണ് റാഷി . ആരാണ് ഈ ജസ്പ്രീത് ഭുറ എന്ന് ചോദിച്ച റാഷി , അദ്ദേഹം ഒരു ക്രിക്കറ്റ് താരമാണെന്ന് അറിയാമെന്നും എന്നാൽ ഇതുവരെ നേരിട്ട് കണ്ടിട്ടു പോലുമില്ലെന്നാണ് പറഞ്ഞത് . അദ്ദേഹത്തിൻറെ വ്യ...",1,False
2,ബോളിവുഡിലെ സൂപ്പർതാരം വെള്ളിത്തിരക്ക് പുറത്ത് മികച്ച പ്രഭാഷകൻ കൂടിയാണ് . ചോദ്യങ്ങളോട് ഷാറൂഖി ൻറെ പ്രതികരണം പല പ്പോഴും പ്രേക്ഷകരെ അത്ഭുതപ്പെടുത്തുാറുണ്ട് . ഇത്തവണ ഷാറൂഖി ൻറെ പ്രതികരണം മാധ്യമപ്രവർത്തകയുടെ ചോദ്യത്തോടായിരുന്നു . ഷാറൂഖിനെ അബദ്ധത്തിൽ സൽമാൻ എന്ന് അഭിസംബോധന ചെയ്യുന്ന മാധ്യമപ്രവർത്തക അത് പിൻവലിച്ച് സോറി പറയു മ്പോഴാണ് പ്രേക്ഷകരെ ഒന്നടങ്കം ചിരിപ്പിക്കുന്ന കിങ് ഖാ ൻറെ പ്രതികരണം വന്നത് . വീഡിയോ കാണാം സോറി പറയുന്ന മാധ്യമ പ്രവർത്തകയുടെ പേര് തെറ്റിച്ചുവിളിച്ച് ഹാസ്യാത്മകമായി ഞാനും അങ്ങേയറ്റം ക്ഷമ പറയുന്നുവെന്നായി താരത്തിൻറെ പ്രതികരണം . ഇതോടെ ചോദ്യവുമായി എത്തിയ മാധ്യമപ്രവർത്തകയും സദസും ഒന്...,1,False
3,"സിനിമ ചിത്രീകരണത്തിനിടെ ബോളിവുഡ് നടി കങ്കണ റണാവത്തിന് മുഖത്ത് വെട്ടേറ്റു . വാൾപയറ്റ് രംഗം ചിത്രീകരിക്കുന്നതിനിടെയാണ് മുഖത്ത് വാൾതലപ്പ് കൊണ്ട് മുറിവുണ്ടായത് . ആശുപത്രിയിൽ പ്രവേശിച്ച കങ്കണയ്ക്ക് അടിയന്തിര ചികിസ്ത നൽകി . പുരികത്തിനിടയിൽ 15 തുന്നലിട്ടതിനാൽ ഒരാഴ്ച ആശുപത്രിയിൽ കഴിയണമെന്നാണ് ഡോക്ടർമാരുടെ നിർദേശം . സ്വാതന്ത്രസമരസേനാനി റാണി ലക്ഷ്മി ഭായിയുടെ ജീവചരിത്ര സിനിമയായ മണികർണിക , റാണി ഓഫ് ഝാൻസി എന്ന ചിത്രത്തിൽ അഭിനയിക്കുന്നതിനിടെയാണ് ബുധനാഴ്ച വൈകിട്ടാണ് കങ്കണയ്ക്ക് പരിക്കേറ്റത് . സഹതാരം നിഹാർ പാണ്ഡ്യയുമായി കങ്കണ വാൾപ്പയറ്റ് നടത്തുന്ന രംഗമായിരുന്നു ചിത്രീകരിച്ചുകൊണ്ടിരുന്നത് . കങ്കണയ്ക്ക് പരി...",1,False
4,"ഇങ്ങനെ അനുമതി ലഭിച്ചാൽ വാട്ട്സ്ആപ്പ് , വൈബർ തുടങ്ങിയ ആപ്പുകളുടെ മോഡലിൽ ഡാറ്റ അധിഷ്ഠിത സൌജന്യ കോളുകൾ നടത്താൻ ടെലികോം കമ്പനികൾക്ക് സാധിക്കും . നിലനിൽ ഇത്തരം ആപ്പുകൾ തങ്ങൾക്ക് വെല്ലുവിളിയാണെന്നും , ഇത് ഇൻറർനെറ്റ് അസമത്വമാണെന്നുമാണ് ടെലികോം കമ്പനികളുടെ നിലപാട് . എന്നാൽ പുതിയ നിലപാടിൽ ജനങ്ങളുടെ പ്രതികരണം എടുത്ത ശേഷമായിരിക്കും അന്തിമ തീരുമാനം എന്നാണ് ട്രായിയുടെ നിലപാട് . ഇതിനായുള്ള നടപടികൾ ഉടൻ പൂർത്തിയാക്കും . പിന്നീട് ലഭിക്കുന്ന അഭിപ്രായങ്ങൾ ഒരു മാസം പഠിച്ച ശേഷമായിരിക്കും ഇത് സംബന്ധിച്ച് സർക്കാറിന് അവസാന റിപ്പോർട്ട് നൽകൂ . നിലവിൽ ടെലികോം കമ്പനികൾക്ക് നേരിട്ട് ഇൻറർനെറ്റ് കോളിംഗ് നൽകാൻ രാജ്യത്തെ ...",3,False


In [ ]:
# --- Option 1: Experimental subset (using 10k training examples) ---
raw_train_df = raw_datasets["train"].select(range(4800)).to_pandas()
raw_valid_df = raw_datasets["validation"].select(range(600)).to_pandas()

# --- Option 2: Full dataset (using the predefined training and validation sets) ---
# raw_train_df = pd.DataFrame(raw_datasets['train'], columns=list(raw_datasets['train'].features.keys()))
# raw_valid_df = pd.DataFrame(raw_datasets['validation'], columns=list(raw_datasets['validation'].features.keys()))

raw_train_df["is_valid"] = False
raw_valid_df["is_valid"] = True

mlnews_df = pd.concat([raw_train_df, raw_valid_df])
print(len(mlnews_df))

5400


In [7]:
mlnews_df.shape

(1200, 3)

In [9]:
labels = raw_datasets['train'].features["label"].names
labels

['business', 'entertainment', 'sports', 'technology']


# Using Hugging Face Training Approach

[Pre Training](https://huggingface.co/docs/transformers/training)

[transformers_multilabel-text-classification-](https://colab.research.google.com/drive/18vy67le2DC-iMJK-AiB0vVKtMRAxmBnB?usp=sharing)

## Tokenize the dataset

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(250))

## Configure the Model, Trainer & Metrics

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=5)

In [16]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  report_to="wandb",  # enable logging to W&B
                                  run_name="mlnews-classifier-lr",  # name of the W&B run (optional)
                                  )

In [19]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
%env WANDB_PROJECT=ml-base

env: WANDB_PROJECT=ml-base


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

## Train & Evaluate

In [24]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, is_valid. If text, is_valid are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 189
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: rajeshmvk (use `wandb login 

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, is_valid. If text, is_valid are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 250
  Batch size = 8


{'eval_loss': 0.4239034354686737, 'eval_accuracy': 0.876, 'eval_runtime': 17.7849, 'eval_samples_per_second': 14.057, 'eval_steps_per_second': 1.799, 'epoch': 1.0}


The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, is_valid. If text, is_valid are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 250
  Batch size = 8


{'eval_loss': 0.44524407386779785, 'eval_accuracy': 0.896, 'eval_runtime': 17.7571, 'eval_samples_per_second': 14.079, 'eval_steps_per_second': 1.802, 'epoch': 2.0}


The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, is_valid. If text, is_valid are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 250
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.5289531350135803, 'eval_accuracy': 0.888, 'eval_runtime': 17.8344, 'eval_samples_per_second': 14.018, 'eval_steps_per_second': 1.794, 'epoch': 3.0}
{'train_runtime': 381.3843, 'train_samples_per_second': 3.933, 'train_steps_per_second': 0.496, 'train_loss': 0.3902987929248305, 'epoch': 3.0}


TrainOutput(global_step=189, training_loss=0.3902987929248305, metrics={'train_runtime': 381.3843, 'train_samples_per_second': 3.933, 'train_steps_per_second': 0.496, 'train_loss': 0.3902987929248305, 'epoch': 3.0})

In [25]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, is_valid. If text, is_valid are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 250
  Batch size = 8


{'eval_loss': 0.5289531350135803, 'eval_accuracy': 0.888, 'eval_runtime': 18.0227, 'eval_samples_per_second': 13.871, 'eval_steps_per_second': 1.776, 'epoch': 3.0}


{'epoch': 3.0,
 'eval_accuracy': 0.888,
 'eval_loss': 0.5289531350135803,
 'eval_runtime': 18.0227,
 'eval_samples_per_second': 13.871,
 'eval_steps_per_second': 1.776}

In [26]:
wandb.finish()

eval/accuracy,▁█▅▅
eval/loss,▁▂██
eval/runtime,▂▁▃█
eval/samples_per_second,▇█▆▁
eval/steps_per_second,▇█▆▁
train/epoch,▁▅███
train/global_step,▁▅███
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


## Share

In [27]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [28]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 69 not upgraded.
Need to get 6,800 kB of archives.
After this operation, 15.3 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.1.2 [6,800 kB]
Fetched 6,800 kB in 1s (11.9 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155339 files and director

In [29]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [30]:
tokenizer.push_to_hub("malayalam_news_classifier",use_temp_dir=True)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/rajeshradhakrishnan/malayalam_news_classifier into local empty directory.
tokenizer config file saved in /tmp/tmp0xiro3mj/tokenizer_config.json
Special tokens file saved in /tmp/tmp0xiro3mj/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


Upload file tokenizer.json:   0%|          | 32.0k/16.3M [00:00<?, ?B/s]

Upload file sentencepiece.bpe.model:   1%|          | 32.0k/4.83M [00:00<?, ?B/s]

remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
To https://huggingface.co/rajeshradhakrishnan/malayalam_news_classifier
   8a646c2..7fa2314  main -> main



'https://huggingface.co/rajeshradhakrishnan/malayalam_news_classifier/commit/7fa23143b3b66cb861649c5063a8fcf43f8e6188'

In [31]:
model.push_to_hub('malayalam_news_classifier',use_temp_dir=True)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/rajeshradhakrishnan/malayalam_news_classifier into local empty directory.


Download file tokenizer.json:   0%|          | 16.6k/16.3M [00:00<?, ?B/s]

Download file sentencepiece.bpe.model:   0%|          | 17.2k/4.83M [00:00<?, ?B/s]

Clean file sentencepiece.bpe.model:   0%|          | 1.00k/4.83M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/16.3M [00:00<?, ?B/s]

Configuration saved in /tmp/tmpo44ke4q1/config.json
Model weights saved in /tmp/tmpo44ke4q1/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 32.0k/1.04G [00:00<?, ?B/s]

remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
To https://huggingface.co/rajeshradhakrishnan/malayalam_news_classifier
   7fa2314..f8aac6c  main -> main



'https://huggingface.co/rajeshradhakrishnan/malayalam_news_classifier/commit/f8aac6cf599a76caa42b3a40e51aac40c69e139f'

## Use it

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("rajeshradhakrishnan/malayalam_news_classifier", num_labels=5)
model = AutoModelForSequenceClassification.from_pretrained("rajeshradhakrishnan/malayalam_news_classifier")


Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [5]:
classes = ["business", "entertainment", "sports", "technology"]

sequence_0 = "കൂടുതൽ സ്ഥലലഭ്യത നൽകുന്ന പുഷ് ബാക്ക് സീറ്റുകൾ , ഓരോ നാല് സീറ്റുകൾ വീതവും ടിവി മോണിറ്റർ സൌകര്യം, മികച്ച ഇന്ധനക്ഷമത എല്ലാറ്റിനുമുപരി സമയലാഭം . "
sequence_1 = "സിനിമ ചിത്രീകരണത്തിനിടെ ബോളിവുഡ് നടി കങ്കണ റണാവത്തിന് മുഖത്ത് വെട്ടേറ്റു . "
sequence_2 = "ഇങ്ങനെ അനുമതി ലഭിച്ചാൽ വാട്ട്സ്ആപ്പ് , വൈബർ തുടങ്ങിയ ആപ്പുകളുടെ മോഡലിൽ ഡാറ്റ അധിഷ്ഠിത സൌജന്യ കോളുകൾ നടത്താൻ ടെലികോം കമ്പനികൾക്ക് സാധിക്കും ."


In [ ]:
# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
output = tokenizer(sequence_0, return_tensors="pt")

classification_logits = model(**output).logits

results = torch.softmax(classification_logits, dim=1).tolist()[0]

for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(results[i] * 100))}%")


In [4]:
from transformers.pipelines import pipeline
classifier= pipeline('sentiment-analysis',
                            model=model,
                            tokenizer=tokenizer,
                            device=0 if torch.cuda.is_available is True else -1)

In [7]:
classifier(sequence_1)

[{'label': 'LABEL_1', 'score': 0.9976699948310852}]

# fastai x hf x blurr 

## Mid Level API

In [10]:
model_cls = AutoModelForSequenceClassification

pretrained_model_name = "xlm-roberta-base" #"eliasedwin7/MalayalamBERT"
config = AutoConfig.from_pretrained(pretrained_model_name)
config.num_labels = len(labels)

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls, config=config)
hf_model.config.problem_type = "multi_label_classification"


print(hf_arch)
print(type(hf_config))
print(type(hf_tokenizer))
print(type(hf_model))

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

xlm_roberta
<class 'transformers.models.xlm_roberta.configuration_xlm_roberta.XLMRobertaConfig'>
<class 'transformers.models.xlm_roberta.tokenization_xlm_roberta_fast.XLMRobertaTokenizerFast'>
<class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaForSequenceClassification'>


In [ ]:
set_seed()
blocks = (TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model, batch_tokenize_kwargs={"labels": labels}), MultiCategoryBlock(encoded=True, vocab=labels))
dblock = DataBlock(blocks=blocks, get_x=ColReader("text"), get_y=ColReader("label"), splitter=RandomSplitter(seed=42))

In [ ]:
dls = dblock.dataloaders(mlnews_df, bs=4, val_bs=8)

In [ ]:
b = dls.one_batch()
len(b), b[0]["input_ids"].shape, b[1].shape

(2, torch.Size([4, 512]), torch.Size([4]))

In [ ]:
b

({'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]]),
  'input_ids': tensor([[     0,  53884, 141655,  ...,  32703, 149794,      2],
          [     0,  54580,   3250,  ..., 126300,   2381,      2],
          [     0,  55381,  23834,  ...,  24197,   2845,      2],
          [     0,   4376,  11300,  ...,      6,  17424,      2]]),
  'labels': TensorCategory([1, 1, 1, 3])},
 TensorCategory([1, 1, 1, 3]))

In [ ]:
dls.show_batch(dataloaders=dls, max_n=5, trunc_at=500)

,text,target
0,"മലയാള സിനിമാ പ്രേക്ഷകർ'ടേക്ക് ഓഫി'നായുള്ള കാത്തിരിപ്പിലാണ്. മാർച്ച് 24നാണ് ടേക്ക് ഓഫ് തീയേറ്ററിലെത്തുക. സോഷ്യൽ മീഡിയയിൽ ചർച്ചയായ ട്രെയിലർ നൽകുന്ന പ്രതീക്ഷ ടേക്ക് ഓഫ് മികച്ച സിനിമാനുഭവമാകുമെന്നാണ്. എഡിറ്റർ മഹേഷ് നാരായണൻ ആദ്യമായി സംവിധാനം ചെയ്യുന്ന സിനിമയിൽ പാർവതി, കുഞ്ചാക്കോ ബോബൻ, ഫഹദ്, ആസിഫ് അലി തുടങ്ങി വൻ താരനിരയുമുണ്ട്. യുവകഥാകൃത്ത് പി വി ഷാജികുമാറും മഹേഷ് നാരായണനും ചേർന്നാണ് ടേക്ക് ഓഫിന്റെ തിരക്കഥയും സംഭാഷണവും രചിച്ചിരിക്കുന്നത്. ടേക്ക് ഓഫിന്റെ വിശേഷങ്ങൾ പി വി ഷാജികുമാർ ഏഷ്യാനെറ്റ് ന്യൂസ് ഓൺല",entertainment
1,റിലയൻസ് ജിയോ ഉപഭോക്താക്കളുടെ മുഴുവൻ വിവരങ്ങളും മറ്റൊരു സ്വകാര്യ വെബ്സൈറ്റിൽ പരസ്യപ്പെടുത്തിയതോടെ വലിയ ആശങ്കയാണ് ഇക്കാര്യത്തിലുള്ളത്. www. magicapk. com എന്ന വെബ്സൈറ്റാണ് ഈ വിവരങ്ങൾ പരസ്യപ്പെടുത്തിയത്. സംഭവം പുറത്തറിഞ്ഞതോടെ അണിയറക്കാർ ഈ വെബ്സൈറ്റ് പിൻവലിച്ച് അന്വേഷണ വഴികൾ അടച്ചു. സ്വന്തം വിശ്വാസ്യത തന്നെ അപകടാവസ്ഥയിലായ സാഹചര്യത്തിൽ റിലയൻസ് തന്നെ ഈ വിഷയത്തിൽ അന്വേഷണം ആരംഭിച്ചിട്ടുണ്ട്. ഫോൺഅരീന ( Fonearena. com ) വെബ്സൈറ്റ് പുറത്തുവിട്ട വാർത്ത പ്രകാരം ഇന്ത്യൻ എക്സ്പ്രസ് നടത്തിയ പരിശോധനയിലും ഇത് ഉപഭ,technology
2,"രാജ്യത്തെ മുൻനിര സ്മാർട്ട് ഫോൺ ബ്രാൻഡായ ഷവോമി അവരുടെ ഏറ്റവും മികച്ച മോഡലായ റെഡ്മി നോട്ട് 6 പ്രോ, എംഐ എൽഇഡി സ്മാർട്ട് ടിവി പ്രോ സീരീസ് എന്നിവ കേരള വിപണിയിൽ അവതരിപ്പിച്ചു. ഓഫ്ലൈൻ വ്യാപാരത്തിൽ കഴിഞ്ഞ ഒരു വർഷം കൊണ്ടുതന്നെ മികച്ച വളർച്ച രേഖപ്പെടുത്തികൊണ്ട് വൻ മുന്നേറ്റമാണ് ഷവോമി കാഴ്ചവെക്കുന്നതെന്ന് കമ്പനി അവകാശപ്പെടുന്നു. 2018 സെപ്റ്റബറിലെ ജിഎഫ്കെ റിപ്പോർട്ട് അനുസരിച്ച് 21 ശതമാനമാണ് കൊച്ചിയിലെ ഷവോമി മാർക്കറ്റ് ഷെയർ. കൊച്ചിയിൽ മാത്രം ഷവോമിക്ക് 99ഓളം പാർട്ണർ സ്റ്റോറുകളും 13വൻകിട റീട്ടെയിൽ ഷോപ്പുകളുമുണ",technology
3,"ദില്ലിക്കെതിരെ നിർണായക രഞ്ജിയിൽ കേരളത്തിന് ഒന്നാം ഇന്നിങ്സ് ലീഡ്. ആദ്യ ഇന്നിങ്സിൽ കേരളം ഉയർത്തിയ 320നനെതിരെ ദില്ലി 139ന് എല്ലാവരും പുറത്തായി. 181 റൺസിന്റെ ലീഡാണ് കേരളം നേടിയത്. ജലജ് സക്സേനയുടെ ആറ് വിക്കറ്റ് പ്രകടനാണ് കേരളത്തിന് ലീഡ് സമ്മാനിച്ചത്. സിജോമോൻ ജോസഫ് രണ്ട് വിക്കറ്റെടുത്തു. 41 റൺസെടുത്ത ജോണ്ടി സിദ്ധുവാണ് ദില്ലിയുടെ ടോപ് സ്കോറർ. തകർച്ചയോടെയായിരുന്നു ദില്ലിയുടെ തുടക്കം. 52 റൺസെടുക്കുന്നതിനിടെ അവർക്ക് മൂന്ന് വിക്കറ്റ് നഷ്ടമായിരുന്നു. സാർതക് രഞ്ജൻ ( 4 ), ഹിതൻ ദലാൽ ( 0 ), ജോണ്ടി സിദ്ധു ( 2 )",sports


In [ ]:
wandb.login()
# start logging a wandb run
wandb.init(project='ml-base')

In [ ]:
set_seed()

model = BaseModelWrapper(hf_model)

learn = Learner(
    dls,
    model,
    opt_func=partial(OptimWrapper, opt=torch.optim.Adam),
    loss_func=PreCalculatedCrossEntropyLoss(),  # CrossEntropyLossFlat(),
    metrics=[accuracy],
    cbs=WandbCallback() # cbs=[BaseModelCallback],
    splitter=blurr_splitter,
)

learn.freeze()

In [ ]:
# try:
#     del learn
#     del inf_learn
#     torch.cuda.empty_cache()
# except:
#     pass

In [ ]:
# learn.summary()

In [ ]:
print(len(learn.opt.param_groups))

3


In [ ]:
learn.lr_find(suggest_funcs=[minimum, steep, valley, slide])

ValueError: ignored

In [ ]:
set_seed()
learn.fit_one_cycle(1, lr_max=1e-3,cbs=WandbCallback())

In [ ]:
wandb.finish()

In [ ]:
learn.show_results(learner=learn, max_n=2, trunc_at=500)

In [ ]:
learn.unfreeze()

In [ ]:
set_seed()
learn.fit_one_cycle(2, lr_max=slice(1e-7, 1e-4),cbs=WandbCallback())

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.show_results(learner=learn, max_n=2, trunc_at=500)

In [ ]:
hf_model.config

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_classification",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

## Using High Level API

In [34]:
learn = BlearnerForSequenceClassification.from_data(
    mlnews_df, pretrained_model_name, text_attr="text", label_attr=labels, dl_kwargs={"bs": 4}
)

learn.loss_func.thresh = 0.02

loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past":

KeyError: ignored

In [ ]:
learn.fit_one_cycle(1, lr_max=1e-2)

In [ ]:
learn.show_results(learner=learn, max_n=2)

In [ ]:
comment = """
2032 ഒളിമ്പിക്‌സിന് ബ്രിസ്‌ബെയ്ന്‍ വേദിയാകും; ഗെയിംസിന് വേദിയാകുന്ന മൂന്നാമത്തെ ഓസ്‌ട്രേലിയന്‍ നഗരം
"""
learn.blurr_predict(comment)

In [ ]:
preds, targs, losses = learn.get_preds(with_loss=True)
preds.shape, targs.shape, losses.shape

## Inference

In [ ]:
export_fname = "ml_multi_class_learn_export"

In [ ]:
learn.export(fname=f"{export_fname}.pkl")

In [ ]:
inf_learn = load_learner(fname=f"{export_fname}.pkl")
inf_learn.blurr_predict("2032 ഒളിമ്പിക്‌സിന് ബ്രിസ്‌ബെയ്ന്‍ വേദിയാകും; ഗെയിംസിന് വേദിയാകുന്ന മൂന്നാമത്തെ ഓസ്‌ട്രേലിയന്‍ നഗരം")